Chapter 2: Functions
===

In [58]:
import re

def parse_beer(beer):
    m = re.match('(.*?)\((.*\))', beer)
    if m:
        return dict(zip(['name', 'location'], m.groups()))
    else:
        raise ValueError('Could not parse beer description: {}'.format(beer))

def read_beers():
    beers = []
    with open('beers.txt', 'r') as f:
        for line in f:
            beers.append(parse_beer(line))
        return beers

beers = read_beers()
beers[:10]

[{'location': 'San Diego, CA)', 'name': 'Alesmith Old Ale '},
 {'location': 'Alpine, CA)', 'name': 'Alpine Captain Stout '},
 {'location': 'Boulder, CO)', 'name': 'Avery White Rascal '},
 {'location': 'Frisco, CO)', 'name': 'Backcountry Brewery Berliner Weisse '},
 {'location': 'San Diego, CA)', 'name': 'Ballast Point Sculpin '},
 {'location': 'San Diego, CA)', 'name': 'Ballast Point Sea Monster '},
 {'location': 'San Diego, CA)', 'name': 'Ballast Point Victory At Sea '},
 {'location': 'Roncole Verdi, Italy)',
  'name': 'Birrificio del Ducato  Beersel Mattina '},
 {'location': 'Roncole Verdi, Italy)',
  'name': 'Birrificio del Ducato  Nuova Mattina '},
 {'location': 'Lurago Marinone (CO))',
  'name': 'Birrificio Italiano V\xc3\x83\xc2\xb9d\xc3\x83\xc2\xb9 '}]

Item 14: Prefer exceptions to returning `None`
===
    Avoid giving special meaning to `None` as return value. Raise exceptions on unexpected behavior to delegate handling.

In [59]:
# No
def parse_beer(beer):
    m = re.match('(.*)\((.*\))', beer)
    if m:
        return dict(zip(['name', 'location'], m.groups()))
    else:
        # Couldn't parse, oh well
        return None

# Better
def parse_beer(beer):
    m = re.match('(.*)\((.*\))', beer)
    if m:
        return dict(zip(['name', 'location'], m.groups()))
    else:
        raise ValueError('Could not parse beer description: {}'.format(beer))

Item 15: Know how closures interact with variable scope
===

In [60]:
def matches_previous(previous=None):
    def f(x):
        result = (x == previous)
        previous = x
        return result
    return f

f = matches_previous()
print(f('a'))
print(f('a'))

UnboundLocalError: local variable 'previous' referenced before assignment

In [ ]:
def matches_previous(previous=None):
    previous = [previous]
    def f(x):
        result = (x == previous[0])
        previous[0] = x
        return result
    return f

f = matches_previous()
print(f('a'))
print(f('a'))

Item 16: Consider generators instead of returning lists
===
Good for concision, memory efficiency, lazy evaluation.

In [ ]:
def read_beers():
    with open('beers.txt', 'r') as f:
        for line in f:
            yield parse_beer(line)

Item 17: Be defensive iterating over arguments
===
Takeaway: avoid passing iterators around -- they iterate once only. Instead pass containers (have __iter__ method)

In [ ]:
def hippest_beer(beers):
    chars = [len(beer['name']) for beer in beers]
    is_local = ['CO' in beer['location'] for beer in beers]
    hip_score = [c * int(l) for c, l in zip(chars, is_local)]
    print hip_score
    return beers[hip_score.index(max(hip_score))]

hippest_beer(read_beers())


In [ ]:
def hippest_beer(beers):
    if iter(beers) is iter(beers):
        raise TypeError('Iterator passed as argument (Should be container instead)')
    chars = [len(beer['name']) for beer in beers]
    is_local = ['CO' in beer['location'] for beer in beers]
    hip_score = [c * int(l) for c, l in zip(chars, is_local)]
    return beers[hip_score.index(max(hip_score))]

hippest_beer(read_beers())


In [ ]:
beer_choice = hippest_beer(list(read_beers()))
beer_choice

Item 18: Reduce visual noise with variable positional arguments
===


In [ ]:
def order(*args):
    print('Hi!')
    if args:
        print('I\'ll have ' + 'and '.join(args) + ', please and thank you.')
    else:
        print('Nothing for me')

order(beer_choice['name'], 'those mac and cheese things')

Item 19: Provide optional behavior with keyword arguments
===
Keyword args:
* facilitate self-documenting code
* can have defaults
* can extend function with optional/alternate behavior

Passing keyword arguments by position generally a no-no

In [ ]:
def my_func(req_arg1, req_arg2, opt_arg1=default1, opt_arg2=default2):
    pass

Item 20: Use `None` and docstrings for dynamic default arguments
===
* Mutables are passed by reference and 
* expressions evaluated when definition parsed


In [ ]:
def f(x, y=[]):
    y.append(x)
    return y
    
print(f(0))
print(f(1, ['a']))
print(f(2))

In [ ]:
def f(x, y=None):
    '''
    Append x to a list
    
    Arguments:
        x: item to append
        y: list to append x to
            if None, will append to empty list
    '''
    if y is None:
        y = []
    y.append(x)
    return y
    
print(f(0))
print(f(1, ['a']))
print(f(2))

Item 21: Enforce clarity with keyword-only arguments
===
Because human nature is sinful, and must be forcibly tamed.

In [ ]:
# python 2
def my_func(req_arg1, req_arg2, **kwargs):
    opt_arg1 = kwargs.pop('opt_arg1', True)
    opt_arg2 = kwargs.pop('opt_arg2', False)
    # ...

my_func('a', 2, False, True)

In [ ]:
my_func('a', 2, opt_arg1=False, opt_arg2=True)

In [ ]:
# python 3
def my_func(req_arg1, req_arg2, *, flag1=default1, opt_arg2=default2):
    pass